# mdr_text 프롬프팅 과정

In [1]:
# mdr_text 프롬프팅 과정
from pydantic import BaseModel, Field, field_validator
from typing import List
from enum import Enum
import ollama
import json
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from typing import Optional
import re

OUTPUT_TEMPLATE = """
Return JSON only with this shape:
{
  "incident_details":{
    "patient_harm":"No Harm|Minor Injury|Serious Injury|Death|Unknown",
    "problem_components":[],
    "incident_summary":""
  },
  "manufacturer_inspection":{
    "defect_confirmed":true|false,
    "defect_type":"Functional Failure|Mechanical/Structural|Electrical/Power|Software/Interface|Alarm/Alert|Sensor/Accuracy|Communication/Connectivity|Labeling/Packaging|Sterility/Contamination|User/Human Factor|Environmental/Compatibility|Other|Unknown",
    "inspection_actions":""
  }
}
"""

# Enum 정의
class PatientHarm(str, Enum):
    NO_HARM = "No Harm"
    MINOR_INJURY = "Minor Injury"
    SERIOUS_INJURY = "Serious Injury"
    DEATH = "Death"
    UNKNOWN = "Unknown"

class DefectType(str, Enum):
    FUNCTIONAL_FAILURE = "Functional Failure"
    MECHANICAL_STRUCTURAL = "Mechanical/Structural"
    ELECTRICAL_POWER = "Electrical/Power"
    SOFTWARE_INTERFACE = "Software/Interface"
    ALARM_ALERT = "Alarm/Alert"
    SENSOR_ACCURACY = "Sensor/Accuracy"
    COMMUNICATION_CONNECTIVITY = "Communication/Connectivity"
    LABELING_PACKAGING = "Labeling/Packaging"
    STERILITY_CONTAMINATION = "Sterility/Contamination"
    USER_HUMAN_FACTOR = "User/Human Factor"
    ENVIRONMENTAL_COMPATIBILITY = "Environmental/Compatibility"
    OTHER = "Other"
    UNKNOWN = "Unknown"

# BaseModel 정의
class IncidentDetails(BaseModel):
    patient_harm: PatientHarm = Field(description="Level of patient harm associated with the incident")
    problem_components: List[str] = Field(
        default_factory=list,
        description="List of problematic component keywords found in the text",
        min_length=0,
        max_length=5
    )
    incident_summary: str = Field(max_length=200, description="Concise summary of the incident")

class ManufacturerInspection(BaseModel):
    defect_confirmed: bool | None = Field(None, description="Whether the defect was confirmed")
    defect_type: DefectType | None = Field(None, description="Type of defect identified during inspection")
    inspection_actions: str | None = Field(None, max_length=200)

class MAUDEExtraction(BaseModel):
    incident_details: IncidentDetails
    manufacturer_inspection: ManufacturerInspection


SYSTEM_INSTRUCTION = """
You are an expert at analyzing FDA MAUDE medical device adverse event reports.

# TASK
Extract structured information from medical device incident reports with high accuracy.

# EXTRACTION RULES

## 1. Patient Harm
Classify the severity of patient harm using these categories:
- **No Harm**: No injury reported
- **Minor Injury**: Temporary/minor injury requiring minimal intervention
- **Serious Injury**: Significant injury requiring medical intervention
- **Death**: Patient death occurred
- **Unknown**: Harm level not specified in report

## 2. Problem Components
List up to 5 specific component keywords mentioned in the report.
Examples: battery, circuit board, sensor, power supply, software, alarm, enclosure, tubing, connector, display, cable, valve, pump, filter

## 3. Incident Summary
Provide a concise factual summary (max 200 characters).

## 4. Defect Confirmed
- Set to **1 (true)** if manufacturer confirmed the defect OR if you can definitively classify the defect type
- Set to **0 (false)** if defect unconfirmed or unclear
- Note: If Defect Type is NOT 'Unknown' or 'Other', this should typically be 1

## 5. Defect Type
Classify the primary defect using the most specific category:

- **Functional Failure**: Device did not perform its intended function
- **Mechanical/Structural**: Physical component breakage, wear, or structural damage
- **Electrical/Power**: Electrical malfunction, power loss, short circuit, or battery issues
- **Software/Interface**: Software bugs, crashes, freezing, or UI malfunctions
- **Alarm/Alert**: Alarm failed to activate, false alarms, or alert system issues
- **Sensor/Accuracy**: Inaccurate readings, sensor malfunction, or calibration issues
- **Communication/Connectivity**: Data transfer issues, wireless connectivity problems
- **Labeling/Packaging**: Incorrect labels, damaged packaging, or missing instructions
- **Sterility/Contamination**: Sterility breach or contamination detected
- **User/Human Factor**: Misuse, user error, or design issues causing user confusion
- **Environmental/Compatibility**: Issues due to temperature, humidity, or device incompatibility
- **Other**: Use only when defect clearly doesn't fit other categories

**Important**: Avoid 'Unknown' and 'Other' whenever possible. Use the 'product problem' field as a reference to help classify, but verify against the full report text as it may contain errors.

## 6. Inspection Actions
Summarize manufacturer's investigation findings or actions taken (max 200 characters).
If no inspection occurred, state "No inspection reported" or "Pending investigation".

# OUTPUT FORMAT
Return valid JSON matching the provided schema. Be precise and factual.
"""

class BatchMAUDEExtractor:
    def __init__(self, model='qwen3:4b', num_workers=8, max_retries=3):
        self.model = model
        self.num_workers = num_workers
        self.max_retries = max_retries

    # def _clean_json_response(self, response_text: str) -> str:
    #     """JSON 응답 정리"""
    #     response_text = response_text.strip()
        
    #     # 마크다운 코드 블록 제거
    #     if response_text.startswith('```'):
    #         # ```json 또는 ``` 제거
    #         response_text = re.sub(r'^```(?:json)?\s*\n?', '', response_text)
    #         response_text = re.sub(r'\n?```\s*$', '', response_text)
        
    #     return response_text.strip()

    def extract_single(self, row, attempt: int = 1) -> dict:
        """단일 레코드 처리 (재시도 로직 포함)"""
        try:
            text = row['mdr_text']
            product_problem = row['product_problems']

            # 프롬프트 생성
            prompt = f"""
# Analyze this FDA MAUDE report and extract structured data.

{text}

Original product problem (detailed defect type): {product_problem}

# You must respond with valid JSON matching this template:
{OUTPUT_TEMPLATE}
"""
            
            if attempt > 1:
                prompt += f"""\n\n
IMPORTANT: This is retry attempt {attempt}. Previous attempts failed. 
- Ensure your response is ONLY valid JSON, no extra text
- Double-check all enum values match exactly
- Verify all required fields are present"""

            # LLM 호출
            response = ollama.generate(
                model=self.model,
                prompt=prompt,
                system=SYSTEM_INSTRUCTION,
                format='json',
                options={'temperature': 0.1}
            )

            # JSON 파싱
            response_text = response['response'].strip()
            if response_text.startswith('```'):
                response_text = response_text.strip('`').replace('json\n', '', 1)
            
            data = json.loads(response_text.strip())
            validated = MAUDEExtraction(**data)
            
            result = validated.model_dump()
            result['_row_id'] = row.name
            result['_success'] = True
            result['_attempts'] = attempt
            return result

        except Exception as e:
            # 재시도 (대기 시간 없이)
            if attempt < self.max_retries:
                return self.extract_single(row, attempt + 1)
            
            # 최종 실패
            return {
                '_row_id': row.name,
                '_success': False,
                '_error': str(e),
                '_attempts': attempt
            }

    def process_batch(self, df: pd.DataFrame, batch_size: int = 10000, 
                     checkpoint_interval: int = 10, 
                     checkpoint_prefix: str = 'checkpoint') -> pd.DataFrame:
        """배치 처리"""
        all_results = []
        num_chunks = (len(df) - 1) // batch_size + 1
        
        print(f"Total: {len(df):,} records, {num_chunks} chunks, {self.num_workers} workers\n")
        
        overall_start = time.time()
        
        for chunk_idx in range(num_chunks):
            start_idx = chunk_idx * batch_size
            end_idx = min((chunk_idx + 1) * batch_size, len(df))
            chunk_df = df.iloc[start_idx:end_idx]
            
            chunk_start = time.time()
            print(f"Chunk {chunk_idx + 1}/{num_chunks}: {start_idx:,}-{end_idx - 1:,}")
            
            chunk_results = []
            with ThreadPoolExecutor(max_workers=self.num_workers) as executor:
                futures = {
                    executor.submit(self.extract_single, row): idx
                    for idx, row in chunk_df.iterrows()
                }
                
                for future in tqdm(as_completed(futures), 
                                 total=len(futures),
                                 ncols=100):
                    chunk_results.append(future.result(timeout=180))
            
            all_results.extend(chunk_results)
            
            # 청크 통계
            elapsed = time.time() - chunk_start
            success = sum(1 for r in chunk_results if r.get('_success'))
            print(f"Success {success}/{len(chunk_results)}, {elapsed:.1f}s, "
                  f"{len(chunk_results)/elapsed:.1f}/s\n")
            
            # 체크포인트
            if (chunk_idx + 1) % checkpoint_interval == 0:
                pd.json_normalize(all_results).to_csv(
                    f'{checkpoint_prefix}_{chunk_idx + 1}.csv', index=False)
                print(f"Checkpoint saved: {checkpoint_prefix}_{chunk_idx + 1}.csv\n")
        
        # 최종 통계
        total_elapsed = time.time() - overall_start
        total_success = sum(1 for r in all_results if r.get('_success'))
        
        print(f"\n{'='*60}")
        print(f"Complete! Time: {total_elapsed/60:.1f}min, "
              f"Success: {total_success/len(all_results)*100:.1f}%")
        print(f"{'='*60}\n")
        
        return pd.json_normalize(all_results)


# # 사용 예시
# if __name__ == "__main__":
#     df = pd.read_csv('your_data.csv')
    
#     extractor = BatchMAUDEExtractor(
#         model='qwen3:8b',
#         num_workers=8,
#         max_retries=3
#     )
    
#     results_df = extractor.process_batch(df, batch_size=10000)
#     results_df.to_csv('maude_results.csv', index=False)

In [1]:
# mdr_text 프롬프팅 과정
from pydantic import BaseModel, Field, field_validator
from typing import List
from enum import Enum
import ollama
import json
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from typing import Optional
import re

OUTPUT_TEMPLATE = """
Return JSON only with this shape:
{
  "incident_details":{
    "patient_harm":"No Harm|Minor Injury|Serious Injury|Death|Unknown",
    "problem_components":[],
    "incident_summary":""
  },
  "manufacturer_inspection":{
    "defect_confirmed":true|false,
    "defect_type":"Functional Failure|Mechanical/Structural|Electrical/Power|Software/Interface|Alarm/Alert|Sensor/Accuracy|Communication/Connectivity|Labeling/Packaging|Sterility/Contamination|User/Human Factor|Environmental/Compatibility|Other|Unknown",
    "inspection_actions":""
  }
}
"""

# Enum 정의
class PatientHarm(str, Enum):
    NO_HARM = "No Harm"
    MINOR_INJURY = "Minor Injury"
    SERIOUS_INJURY = "Serious Injury"
    DEATH = "Death"
    UNKNOWN = "Unknown"

class DefectType(str, Enum):
    FUNCTIONAL_FAILURE = "Functional Failure"
    MECHANICAL_STRUCTURAL = "Mechanical/Structural"
    ELECTRICAL_POWER = "Electrical/Power"
    SOFTWARE_INTERFACE = "Software/Interface"
    ALARM_ALERT = "Alarm/Alert"
    SENSOR_ACCURACY = "Sensor/Accuracy"
    COMMUNICATION_CONNECTIVITY = "Communication/Connectivity"
    LABELING_PACKAGING = "Labeling/Packaging"
    STERILITY_CONTAMINATION = "Sterility/Contamination"
    USER_HUMAN_FACTOR = "User/Human Factor"
    ENVIRONMENTAL_COMPATIBILITY = "Environmental/Compatibility"
    OTHER = "Other"
    UNKNOWN = "Unknown"

# BaseModel 정의
class IncidentDetails(BaseModel):
    patient_harm: PatientHarm = Field(description="Level of patient harm associated with the incident")
    problem_components: List[str] = Field(
        default_factory=list,
        description="List of problematic component keywords found in the text",
        min_length=0,
        max_length=5
    )
    incident_summary: str = Field(max_length=200, description="Concise summary of the incident")

class ManufacturerInspection(BaseModel):
    defect_confirmed: bool | None = Field(None, description="Whether the defect was confirmed")
    defect_type: DefectType | None = Field(None, description="Type of defect identified during inspection")
    inspection_actions: str | None = Field(None, max_length=200)

class MAUDEExtraction(BaseModel):
    incident_details: IncidentDetails
    manufacturer_inspection: ManufacturerInspection


SYSTEM_INSTRUCTION = """You are an expert at analyzing FDA MAUDE medical device adverse event reports.

Extract structured information from medical device incident reports with high accuracy.

# EXTRACTION RULES

## Patient Harm
- No Harm: No injury reported
- Minor Injury: Temporary/minor injury requiring minimal intervention
- Serious Injury: Significant injury requiring medical intervention
- Death: Patient death occurred
- Unknown: Harm level not specified in report

## Problem Components
List up to 5 specific component keywords (battery, circuit board, sensor, power supply, software, alarm, enclosure, tubing, connector, display, cable, valve, pump, filter)

## Incident Summary
Provide a concise factual summary (max 200 characters).

## Defect Confirmed
Set to true if manufacturer confirmed defect OR if you can definitively classify defect type. Set to false if unconfirmed.

## Defect Type:
**CLASSIFICATION RULES:**
- Analyze the ENTIRE MDR text to identify the ROOT CAUSE
- Use product_problem as supporting evidence only
- Choose the MOST SPECIFIC category for the primary failure
- Use "Unknown" only when MDR lacks sufficient detail
- Use "Other" only when defect doesn't fit any specific category

**Categories with Keywords:**

### 1. Functional Failure
Device failed to perform its core function
**Keywords**: didn't work, stopped working, not functioning, no output, inoperative
**Examples**: Pump stopped delivering, ventilator failed, defibrillator didn't shock

### 2. Mechanical/Structural
Physical breakage or structural damage
**Keywords**: broke, cracked, fractured, separated, detached, loose, wear, tear, leak
**Examples**: Catheter tip broke off, housing cracked, component detached

### 3. Electrical/Power
Electrical failures, power issues, thermal events
**Keywords**: battery, power, overheating, short circuit, thermal, burn, won't charge
**Examples**: Battery drained, device overheated, power failed, burning smell

### 4. Software/Interface
Software errors, display issues, UI problems
**Keywords**: software, display, frozen, crash, error message, unresponsive, reboot
**Examples**: Display error, software crashed, touchscreen unresponsive

### 5. Alarm/Alert
Alarm system failures or false alarms
**Keywords**: alarm, alert, no alarm, false alarm, alarm didn't sound
**Examples**: Critical alarm failed, false alarms, alarm couldn't silence

### 6. Sensor/Accuracy
Sensor failures, measurement errors, inaccurate readings
**Keywords**: inaccurate, incorrect reading, sensor, calibration, wrong result
**Examples**: Glucose meter incorrect, pressure sensor false reading

### 7. Communication/Connectivity
Data transfer failures, connectivity problems
**Keywords**: communication, wireless, Bluetooth, connection lost, data transfer
**Examples**: Lost wireless connection, data didn't transfer, pairing failed

### 8. Labeling/Packaging
Incorrect labeling, packaging defects, documentation errors
**Keywords**: label, packaging, wrong size, mislabeled, missing instructions
**Examples**: Wrong size label, missing instructions, damaged packaging

### 9. Sterility/Contamination
Sterility breaches, contamination, foreign materials
**Keywords**: sterile, contamination, foreign material, particle, debris, residue
**Examples**: Non-sterile device, foreign particles, residue found

### 10. User/Human Factor
Design-induced user errors, usability problems (NOT simple user mistakes)
**Keywords**: user error, confusing, difficult to use, unclear, poor design
**Examples**: Confusing interface, similar-looking connectors, hard to read

### 11. Environmental/Compatibility
Environmental factors or device incompatibility
**Keywords**: temperature, humidity, environment, compatibility, incompatible
**Examples**: High humidity malfunction, incompatible accessory, temperature extreme

### 12. Other
**ONLY use when**: The defect is clearly described BUT doesn't fit ANY of the 11 specific categories above
**NOT for**: Vague descriptions (use Unknown instead), or cases that partially match a category (choose the closest match)
**Rare examples**: Pure cosmetic defects with no functional impact, shipping/handling damage unrelated to device design

### 13. Unknown
**ONLY use when**: MDR text provides insufficient information to determine the type of defect
**Indicators**: "unknown cause", "under investigation with no findings yet", "incident occurred but no defect identified", vague description with no specifics
**NOT for**: Cases where you can infer defect type from symptoms described

## Inspection Actions
Summarize manufacturer's investigation findings (max 200 characters). If no inspection, state "No inspection reported".

Return valid JSON matching the provided schema."""

class BatchMAUDEExtractor:
    def __init__(self, model='qwen3:4b', num_workers=8, max_retries=3):
        self.model = model
        self.num_workers = num_workers
        self.max_retries = max_retries

    def extract_single(self, row, attempt: int = 1) -> dict:
        """단일 레코드 처리 (재시도 로직 포함)"""
        try:
            text = row['mdr_text']
            product_problem = row['product_problems']

            # 프롬프트 생성
            prompt = f"""# Analyze this FDA MAUDE report and extract structured data.

{text}

Original product problem: {product_problem}

{OUTPUT_TEMPLATE}"""
            
            if attempt > 1:
                prompt += "\n\nRETRY: Return ONLY valid JSON with exact enum values."

            # LLM 호출
            response = ollama.generate(
                model=self.model,
                prompt=prompt,
                system=SYSTEM_INSTRUCTION,
                format='json',
                options={'temperature': 0.1}
            )

            # JSON 파싱 (간소화)
            response_text = response['response'].strip()
            if response_text.startswith('```'):
                response_text = response_text.strip('`').strip()
                if response_text.startswith('json'):
                    response_text = response_text[4:].strip()
            
            data = json.loads(response_text)
            validated = MAUDEExtraction(**data)
            
            result = validated.model_dump()
            result['_row_id'] = row.name
            result['_success'] = True
            result['_attempts'] = attempt
            return result

        except Exception as e:
            if attempt < self.max_retries:
                return self.extract_single(row, attempt + 1)
            
            return {
                '_row_id': row.name,
                '_success': False,
                '_error': str(e)[:200],
                '_attempts': attempt
            }

    def process_batch(self, df: pd.DataFrame, batch_size: int = 10000, 
                     checkpoint_interval: int = 10, 
                     checkpoint_prefix: str = 'checkpoint') -> pd.DataFrame:
        """배치 처리"""
        all_results = []
        num_chunks = (len(df) - 1) // batch_size + 1
        
        print(f"Total: {len(df):,} records, {num_chunks} chunks, {self.num_workers} workers\n")
        
        overall_start = time.time()
        
        for chunk_idx in range(num_chunks):
            start_idx = chunk_idx * batch_size
            end_idx = min((chunk_idx + 1) * batch_size, len(df))
            chunk_df = df.iloc[start_idx:end_idx]
            
            chunk_start = time.time()
            print(f"Chunk {chunk_idx + 1}/{num_chunks}: {start_idx:,}-{end_idx - 1:,}")
            
            chunk_results = []
            with ThreadPoolExecutor(max_workers=self.num_workers) as executor:
                futures = [executor.submit(self.extract_single, row) 
                          for _, row in chunk_df.iterrows()]
                
                for future in tqdm(as_completed(futures), 
                                 total=len(futures),
                                 ncols=80):
                    chunk_results.append(future.result(timeout=180))
            
            all_results.extend(chunk_results)
            
            # 청크 통계
            elapsed = time.time() - chunk_start
            success = sum(1 for r in chunk_results if r.get('_success'))
            print(f"Success {success}/{len(chunk_results)}, {elapsed:.1f}s, "
                  f"{len(chunk_results)/elapsed:.1f}/s\n")
            
            # 체크포인트
            if (chunk_idx + 1) % checkpoint_interval == 0:
                pd.json_normalize(all_results).to_csv(
                    f'{checkpoint_prefix}_{chunk_idx + 1}.csv', index=False)
                print(f"Checkpoint: {checkpoint_prefix}_{chunk_idx + 1}.csv\n")
        
        # 최종 통계
        total_elapsed = time.time() - overall_start
        total_success = sum(1 for r in all_results if r.get('_success'))
        
        print(f"\n{'='*60}")
        print(f"Complete! {total_elapsed/60:.1f}min, "
              f"Success: {total_success/len(all_results)*100:.1f}%")
        print(f"{'='*60}\n")
        
        return pd.json_normalize(all_results)

In [2]:
import sys
from pathlib import Path

# 상대 경로 사용
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'

# 맨 앞에 추가
sys.path.insert(0, str(PROJECT_ROOT))

# Python 내장 code 모듈 캐시만 임시 제거
if 'code' in sys.modules:
    del sys.modules['code']

# 이제 import
from code.loading import DataLoader

loader2 = DataLoader(
    start=2020,
    end=2025,
    output_file=DATA_DIR / 'maude_sample.parquet',
    max_workers=4
)

In [3]:
adapter = 'polars'
polars_kwargs = {
    'use_statistics': True,
    'parallel': 'auto',
    'low_memory': False,
    'rechunk': False,
    'cache': True,
}

df = loader2.load(adapter=adapter, **polars_kwargs)


📖 /Users/t2023-m0103/Desktop/sparta/final_project/data/maude_sample.parquet 로딩 중... (adapter=polars)


In [4]:
import polars as pl

In [5]:
MDR_COLS = [f'mdr_text_{i}_text' for i in range(5)]

# mdr_lf = maude_lf.select(pl.col(MDR_COLS))

mdr_concat_lf = df.filter((pl.col("mdr_text_0_text").is_not_null()) & (pl.col("device_0_openfda_device_class")=="3")).with_columns(
    pl.concat_str(MDR_COLS, separator='\n', ignore_nulls=True).alias('mdr_text')
) # ignore nulls는 연결하는 열 중 하나라도 null인 경우 False면 전부 null 됨

mdr_concat_lf.select('mdr_text').head().collect()

mdr_text
str
"""PRODUCT COMPLAINT (B)(4) THIS …"
"""IT WAS REPORTED THAT THE PATIE…"
"""IT WAS REPORTED THROUGH PATIEN…"
"""BLOCK B3: APPROXIMATED BASED O…"
"""IT WAS REPORTED THAT POST-IMPL…"


In [6]:
row_count = mdr_concat_lf.select(pl.len()).collect().item()
print(f"행 수: {row_count}")

행 수: 565136


In [7]:
# df2 = df.filter(pl.col("mdr_text_0_text").is_not_null()).head(10).collect().to_pandas()

sampled_lf = mdr_concat_lf.select(
    pl.all().sample(
        n=1000,
        with_replacement=False,
        shuffle=True, # Shuffle the order of sampled rows
        seed=4242
    )
).collect().to_pandas()

# df2 = (
#     mdr_concat_lf
#     .collect()
#     .sample(n=1000, seed=42)  # seed로 재현 가능
#     .to_pandas()
# )

sampled_lf.head()

,adverse_event_flag,date_added,date_changed,date_facility_aware,date_manufacturer_received,date_of_event,date_received,date_report,date_report_to_fda,date_report_to_manufacturer,...,reporter_country_code,reporter_occupation_code,reporter_state_code,reprocessed_and_reused_flag,single_use_flag,source_type,summary_report_flag,suppl_dates_fda_received,suppl_dates_mfr_received,type_of_report
0,N,20240930,20241007,None,20240909,20240909,20240930,20240930,None,None,...,SP,003,None,N,Y,"['Foreign', 'Consumer']",N,None,None,['Initial submission']
1,N,20240627,20240708,None,20240611,20240611,20240627,20240627,None,None,...,US,003,None,N,N,['Consumer'],N,None,None,['Initial submission']
2,Y,20241001,20241108,None,20240925,20240908,20241001,20241001,None,None,...,None,OTHER,None,N,N,['Distributor'],N,None,None,['Initial submission']
3,N,20240509,20240624,None,20240420,20240420,20240509,20240509,None,None,...,US,003,None,N,N,['Consumer'],N,None,None,['Initial submission']
4,N,20240531,20241101,None,20240526,20240526,20240531,20241028,None,None,...,None,003,None,N,N,"['Foreign', 'Consumer']",N,10/28/2024,10/03/2024,"['Initial submission', 'Followup']"


In [ ]:
start_time = time.time()

extractor = BatchMAUDEExtractor(model='qwen3:4b', num_workers=8, max_retries=2)
results_df = extractor.process_batch(sampled_lf, batch_size=10)

elapsed = time.time() - start_time
print(f"Elapsed time: {elapsed:.2f} seconds")

Total: 1,000 records, 100 chunks, 8 workers

Chunk 1/100: 0-9


100%|███████████████████████████████████████████████████████████████| 10/10 [03:42<00:00, 22.23s/it]


Success 0/10, 222.4s, 0.0/s

Chunk 2/100: 10-19


100%|███████████████████████████████████████████████████████████████| 10/10 [03:40<00:00, 22.05s/it]


Success 0/10, 220.5s, 0.0/s

Chunk 3/100: 20-29


...

100%|███████████████████████████████████████████████████████████████| 10/10 [03:23<00:00, 20.36s/it]


Success 0/10, 203.7s, 0.0/s

Chunk 29/100: 280-289


100%|███████████████████████████████████████████████████████████████| 10/10 [02:59<00:00, 17.99s/it]


Success 0/10, 180.0s, 0.1/s

Chunk 30/100: 290-299


 80%|███████████████████████████████████████████████████▏            | 8/10 [02:50<00:25, 12.63s/it]

In [ ]:
results_df # 가끔 실패하는 경우가 있음. _success가 False로 나옴

,_row_id,_success,_error,_attempts
0,0,False,model 'qwen3:4b' not found (status code: 404),2
1,2,False,model 'qwen3:4b' not found (status code: 404),2
2,3,False,model 'qwen3:4b' not found (status code: 404),2
3,1,False,model 'qwen3:4b' not found (status code: 404),2
4,4,False,model 'qwen3:4b' not found (status code: 404),2
...,...,...,...,...
995,992,False,model 'qwen3:4b' not found (status code: 404),2
996,997,False,model 'qwen3:4b' not found (status code: 404),2
997,993,False,model 'qwen3:4b' not found (status code: 404),2
998,998,False,model 'qwen3:4b' not found (status code: 404),2


In [ ]:
# 필요한 열만 선택 후 열 이름 변경
result_df2 = results_df[[
    'incident_details.patient_harm',
    'incident_details.problem_components',
    'incident_details.incident_summary',
    'manufacturer_inspection.defect_confirmed',
    'manufacturer_inspection.defect_type',
    'manufacturer_inspection.inspection_actions'
    ]]

result_df2 = result_df2.rename(columns={
    'incident_details.patient_harm': 'patient_harm',
    'incident_details.problem_components': 'problem_components',
    'incident_details.incident_summary': 'incident_summary',
    'manufacturer_inspection.defect_confirmed': 'defect_confirmed',
    'manufacturer_inspection.defect_type': 'defect_type',
    'manufacturer_inspection.inspection_actions': 'inspection_actions'
})

result_df2.head(5)

KeyError: "None of [Index(['incident_details.patient_harm', 'incident_details.problem_components',\n       'incident_details.incident_summary',\n       'manufacturer_inspection.defect_confirmed',\n       'manufacturer_inspection.defect_type',\n       'manufacturer_inspection.inspection_actions'],\n      dtype='object')] are in the [columns]"

In [ ]:
# df_concat = pd.concat([sampled_lf, result_df2], axis=1)
# df_concat[['mdr_text', 'patient_harm', 'defect_type']]

,mdr_text,patient_harm,defect_type
0,SELECT PATIENT INFORMATION CANNOT BE PROVIDED ...,PatientHarm.NO_HARM,DefectType.SENSOR_ACCURACY
1,CURRENTLY IT IS UNKNOWN WHETHER OR NOT THE DEV...,PatientHarm.NO_HARM,DefectType.COMMUNICATION_CONNECTIVITY
2,A PATIENT RECEIVED AN INAPPROPRIATE SHOCK. AF ...,PatientHarm.NO_HARM,DefectType.COMMUNICATION_CONNECTIVITY
3,IT WAS REPORTED TO MEDTRONIC MINIMED THAT THE ...,PatientHarm.MINOR_INJURY,DefectType.SENSOR_ACCURACY
4,AN ATTEMPT TO REPRODUCE THE REPORTED EVENT WAS...,PatientHarm.NO_HARM,DefectType.COMMUNICATION_CONNECTIVITY
...,...,...,...
995,IT WAS REPORTED TO MEDTRONIC MINIMED THAT THE ...,PatientHarm.NO_HARM,DefectType.FUNCTIONAL_FAILURE
996,IT WAS REPORTED THAT THE SUBCUTANEOUS IMPLANTA...,PatientHarm.NO_HARM,DefectType.SENSOR_ACCURACY
997,"HEALTHCARE PROFESSIONAL REPORTED ""DEFLATION."" ...",PatientHarm.UNKNOWN,DefectType.OTHER
998,IT WAS REPORTED THAT THIS CARDIAC RESYNCHRONIZ...,PatientHarm.NO_HARM,DefectType.ELECTRICAL_POWER


In [ ]:
# df_concat.to_csv('maude_extracted_sample2.csv', index=False)

# 프롬프팅 관련 문제
1. 예외 처리가 없어서 llm 다차원분리에 실패하더라도 그대로 그 행이 빈 채로 넘어가짐 <- 개선 필요
2. 드는 시간이 너무 많이 걸려서 프롬프트를 좀 크기를 단축시켜야 됨.
    * 실제로는 여기서 더 단축시키기가 힘듦.
3. system prompt는 더 길어져도 한번만 들어가기 때문에 부담 없이 길게 할 수 있음
    * 여기가 주로 만져야 되는 부분(퀄리티 상승을 위해서)